# Combine: Intercity Passenger Rail Service Station Performance Metrics

This notebook reads a series of Excel files containing [Amtrak](https://www.amtrak.com/home.html)
station performance metrics, combines the data into a single [pandas](https://pandas.pydata.org/)
`DataFrame`, and writes the combined dataset to a CSV file for follow up cleaning, manipulation,
and analysis.

### Variable names

A number of variable names in this project leverage the following abbreviations. The naming
strategy is to strike a balance between brevity and readability:

* `amtk`: Amtrak (reporting mark)
* `chrt`: chart
* `cols`: columns
* `const`: constant
* `cwd`: current working directory
* `eb`: eastbound direction of travel
* `lm`: linear model
* `mi`: miles
* `mm`: minutes (ISO 8601)
* `nb`: northbound direction of travel
* `psgr`: passenger
* `qtr`: quarter
* `rte`: route
* `sb`: southbound direction of travel
* `stats`: summary statistics
* `stn`: station
* `stns`: stations
* `svc`: service
* `trn`: train
* `wb`: westbound direction of travel

In [1]:
import numpy as np
import pandas as pd
import pathlib as pl
import tomllib as tl

import fra_amtrak.amtk_frame as frm

## 1.0 Read files

### 1.1 Create paths

Instantiate instances of `pathlib.Path` to represent absolute paths to the `data/raw` and
`data/interim` directories.

In [2]:
parent_path = pl.Path.cwd()  # current working directory
parent_path

data_raw_path = parent_path.joinpath("data", "raw")
data_interim_path = parent_path.joinpath("data", "interim")

### 1.2 Load constants

Load a companion [TOML](https://toml.io/en/) file named `notebook.toml` containing constants.

In [3]:
filepath = parent_path.joinpath("notebook.toml")
with open(filepath, "rb") as file_obj:
    const = tl.load(file_obj)

# Access constants
COLS = const["columns"]

### 1.3 Read data files

The FRA's station performance filenames are inconsistent (review the files in the `data/raw`
directory). The [`Path.glob()`](https://docs.python.org/3/library/pathlib.html#pathlib.Path.glob)
method is used to retrieve the otherwise irregular filepaths via pattern matching.

In [4]:
# Check filepaths (this can also be done with the os or glob modules)
filepaths = data_raw_path.glob("*Station%20Performance*.xlsx")
filepaths = [filepath for filepath in filepaths if filepath.is_file()]

print(f"filepaths (n={len(filepaths)})")

filepaths (n=12)


Below are the preferred data types for each column in the dataset:

In [5]:
# Specify dtypes
dtypes = {
    "Fiscal Year": np.int16,
    "Fiscal Quarter": np.int8,
    "Service Line": "string",
    "Service": "string",
    "Sub Service": "string",
    "Train Number": np.int16,
    "Arrival Station Code": "string",
    "Arrival Station Name": "string",
    "Total Detraining Customers": np.int32,
    "Late Detraining Customers": np.int32,
    # "Avg Min Late (Lt CS)": np.int32,  # Triggers ValueError: invalid literal for int() with base 10: '--'
    # "Avg Min Late (Lt C)": np.int32,  # Triggers ValueError: invalid literal for int() with base 10: '--'
}


### 1.4 Combine `DataFrames` [1 pt]

The next task is to use pandas to read the station peformance `*.xlsx` files located in the
`data/raw` directory into a set of unnamed `DataFrame` objects, specifying specific column `dtypes`
for each using a dictionary, and ignoring the indexes.  The `DataFrame` objects are then combined
into a single `DataFrame` object named `stations`.

Note: this operation can be performed in a single line of code, although length considerations will
dictate the use of multiple lines.

In [6]:
# YOUR CODE HERE
dfs = [pd.read_excel(path, dtype=dtypes) for path in filepaths]
stations = pd.concat(dfs, ignore_index=True)
stations

,Unnamed: 0,Fiscal Year,Fiscal Quarter,Service Line,Service,Sub Service,Train Number,Arrival Station Code,Arrival Station Name,Total Detraining Customers,Late Detraining Customers,Avg Min Late (Lt CS),Avg Min Late (Lt C)
0,NaN,2024,1,Long Distance,Auto Train,Auto Train,52,LOR,"Lorton (Auto Train), Virginia",25801,5199,71,NaN
1,NaN,2024,1,Long Distance,Auto Train,Auto Train,53,SFA,"Sanford (Auto Train), Florida",38507,11140,41,NaN
2,NaN,2024,1,Long Distance,California Zephyr,California Zephyr,5,BRL,"Burlington, Iowa",651,316,40,NaN
3,NaN,2024,1,Long Distance,California Zephyr,California Zephyr,5,COX,"Colfax, California",475,336,97,NaN
4,NaN,2024,1,Long Distance,California Zephyr,California Zephyr,5,CRN,"Creston, Iowa",155,113,42,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68407,NaN,2022,2,State Supported,Vermonter,Vermonter,57,WAS,"Washington, District of Columbia",3584,423,34,NaN
68408,NaN,2022,2,State Supported,Vermonter,Vermonter,57,WIL,"Wilmington, Delaware",538,114,25,NaN
68409,NaN,2022,2,State Supported,Vermonter,Vermonter,57,WNL,"Windsor Locks, Connecticut",41,10,17,NaN
68410,NaN,2022,2,State Supported,Vermonter,Vermonter,57,WNM,"Windsor, Vermont",7,4,18,NaN


In [7]:
# Hidden tests are within this cell

In [8]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68412 entries, 0 to 68411
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  0 non-null      float64
 1   Fiscal Year                 68412 non-null  int16  
 2   Fiscal Quarter              68412 non-null  int8   
 3   Service Line                68412 non-null  string 
 4   Service                     68412 non-null  string 
 5   Sub Service                 68412 non-null  string 
 6   Train Number                68412 non-null  int16  
 7   Arrival Station Code        68412 non-null  string 
 8   Arrival Station Name        68412 non-null  string 
 9   Total Detraining Customers  68412 non-null  int32  
 10  Late Detraining Customers   68412 non-null  int32  
 11  Avg Min Late (Lt CS)        62600 non-null  object 
 12  Avg Min Late (Lt C)         5656 non-null   object 
dtypes: float64(1), int16(2), int32(

In [9]:
stations.head()

,Unnamed: 0,Fiscal Year,Fiscal Quarter,Service Line,Service,Sub Service,Train Number,Arrival Station Code,Arrival Station Name,Total Detraining Customers,Late Detraining Customers,Avg Min Late (Lt CS),Avg Min Late (Lt C)
0,NaN,2024,1,Long Distance,Auto Train,Auto Train,52,LOR,"Lorton (Auto Train), Virginia",25801,5199,71,NaN
1,NaN,2024,1,Long Distance,Auto Train,Auto Train,53,SFA,"Sanford (Auto Train), Florida",38507,11140,41,NaN
2,NaN,2024,1,Long Distance,California Zephyr,California Zephyr,5,BRL,"Burlington, Iowa",651,316,40,NaN
3,NaN,2024,1,Long Distance,California Zephyr,California Zephyr,5,COX,"Colfax, California",475,336,97,NaN
4,NaN,2024,1,Long Distance,California Zephyr,California Zephyr,5,CRN,"Creston, Iowa",155,113,42,NaN


## 2.0 Drop "Unnamed" column

Each Excel spreadsheet features an unnamed first column consisting of no row values. Drop the column.

In [10]:
# Print unique values in Unnamed column
mask = stations.columns[stations.columns.str.contains("^Unnamed")]
print(f"Unnamed columns (n={len(mask)}):")

# Print unique values in unnamed column
unnamed_unique_values = stations[mask].apply(lambda x: x.unique())
print(f"unnamed_unique_values = {unnamed_unique_values}")

# Drop Unnamed column
stations = stations.loc[:, ~stations.columns.str.contains("^Unnamed")]
stations.head()

Unnamed columns (n=1):
unnamed_unique_values =    Unnamed: 0
0         NaN


,Fiscal Year,Fiscal Quarter,Service Line,Service,Sub Service,Train Number,Arrival Station Code,Arrival Station Name,Total Detraining Customers,Late Detraining Customers,Avg Min Late (Lt CS),Avg Min Late (Lt C)
0,2024,1,Long Distance,Auto Train,Auto Train,52,LOR,"Lorton (Auto Train), Virginia",25801,5199,71,NaN
1,2024,1,Long Distance,Auto Train,Auto Train,53,SFA,"Sanford (Auto Train), Florida",38507,11140,41,NaN
2,2024,1,Long Distance,California Zephyr,California Zephyr,5,BRL,"Burlington, Iowa",651,316,40,NaN
3,2024,1,Long Distance,California Zephyr,California Zephyr,5,COX,"Colfax, California",475,336,97,NaN
4,2024,1,Long Distance,California Zephyr,California Zephyr,5,CRN,"Creston, Iowa",155,113,42,NaN


## 3.0 Transform mixed type columns

Check columns for the presence of mixed types. The goal is to avoid triggering a `DtypeWarning` when
reading the combined dataset into a `DataFrame` in subsequent notebooks.

```commandline
DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  stations = pd.read_csv(filepath, dtype=dytpes)
```

Suspect columns include:

* Avg Min Late (Lt CS)
* Avg Min Late (Lt C)

If strings are present in these columns, convert them to `NaN` and then recast the columns to
`np.float32`.

### 3.1 Traverse the columns

In [11]:
# Traverse data frame to detect data types
for column in stations.columns:
    print(f"{column}: ", pd.api.types.infer_dtype(stations[column]))

Fiscal Year:  integer
Fiscal Quarter:  integer
Service Line:  string
Service:  string
Sub Service:  string
Train Number:  integer
Arrival Station Code:  string
Arrival Station Name:  string
Total Detraining Customers:  integer
Late Detraining Customers:  integer
Avg Min Late (Lt CS):  mixed-integer
Avg Min Late (Lt C):  string


### 3.2 Identify the non-numeric values


Call the function `frm.find_non_numeric_values()` and return the non-numeric values polluting the
"Avg Min Late" columns.

In [12]:
non_numeric_values = {
    column: frm.find_non_numeric_values(stations, column) for column in stations.columns[-2:]
}
non_numeric_values

{'Avg Min Late (Lt CS)': ['--', nan], 'Avg Min Late (Lt C)': [nan, '--']}

Return a count of the string values (`"--"`) in the "Avg Min Late" columns.

In [24]:
lt_cs_dashes_count = stations.loc[:, COLS["avg_mm_late_cs"]].str.contains("--").sum()
print(f"Lt CS dashes count = {lt_cs_dashes_count}")

lt_c_dashes_count = stations.loc[:, COLS["avg_mm_late_c"]].str.contains("--").sum()
print(f"Lt C dashes count = {lt_c_dashes_count}")

Lt CS dashes count = 9895
Lt C dashes count = 988


0        71
1        41
2        40
3        97
4        42
         ..
68407    34
68408    25
68409    17
68410    18
68411    17
Name: Avg Min Late (Lt CS), Length: 68412, dtype: object

### 3.1 Convert dashes to `NaN` [1 pt]

Convert dashes (`--`) in both columns to `NaN` and then convert the columns to `np.float32`.

In [30]:
stations[COLS["avg_mm_late_cs"]] = stations.loc[:, COLS["avg_mm_late_cs"]].replace("--", np.nan).astype(np.float32)
stations[COLS["avg_mm_late_c"]] = stations.loc[:, COLS["avg_mm_late_c"]].replace("--", np.nan).astype(np.float32)

In [ ]:
# Hidden tests are within this cell

Confirm the conversion by checking the `dtype` of each column in `stations`. Note the reduction in memory usage as a result of careful type selection.

In [31]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68412 entries, 0 to 68411
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Fiscal Year                 68412 non-null  int16  
 1   Fiscal Quarter              68412 non-null  int8   
 2   Service Line                68412 non-null  string 
 3   Service                     68412 non-null  string 
 4   Sub Service                 68412 non-null  string 
 5   Train Number                68412 non-null  int16  
 6   Arrival Station Code        68412 non-null  string 
 7   Arrival Station Name        68412 non-null  string 
 8   Total Detraining Customers  68412 non-null  int32  
 9   Late Detraining Customers   68412 non-null  int32  
 10  Avg Min Late (Lt CS)        52705 non-null  float32
 11  Avg Min Late (Lt C)         4668 non-null   float32
dtypes: float32(2), int16(2), int32(2), int8(1), string(5)
memory usage: 4.0 MB


## 4.0 Sort data [1 pt]

Sort `stations` by its first seven (`7`) columns, ordered as follows:

1. "Fiscal Year" (descending)
2. "Fiscal Quarter" (descending)
3. "Service Line" (ascending)
4. "Service" (ascending)
5. "Sub Service" (ascending)
6. "Train Number" (ascending)
7. "Arrival Station Code" (ascending)

In [32]:
stations.sort_values(by=["Fiscal Year", "Fiscal Quarter", "Service Line", "Service", "Sub Service", "Train Number", "Arrival Station Code"], 
                     ascending=[False, False, True, True, True, True, True],
                     inplace=True)

,Fiscal Year,Fiscal Quarter,Service Line,Service,Sub Service,Train Number,Arrival Station Code,Arrival Station Name,Total Detraining Customers,Late Detraining Customers,Avg Min Late (Lt CS),Avg Min Late (Lt C)
17506,2024,3,Long Distance,Auto Train,Auto Train,52,LOR,"Lorton (Auto Train), Virginia",42445,23316,95.0,NaN
17507,2024,3,Long Distance,Auto Train,Auto Train,53,SFA,"Sanford (Auto Train), Florida",28034,18439,91.0,NaN
17508,2024,3,Long Distance,California Zephyr,California Zephyr,5,BRL,"Burlington, Iowa",557,223,54.0,NaN
17509,2024,3,Long Distance,California Zephyr,California Zephyr,5,COX,"Colfax, California",508,326,99.0,NaN
17510,2024,3,Long Distance,California Zephyr,California Zephyr,5,CRN,"Creston, Iowa",205,144,67.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
57241,2021,4,State Supported,Vermonter,Vermonter,57,WAS,"Washington, District of Columbia",5191,187,37.0,NaN
57242,2021,4,State Supported,Vermonter,Vermonter,57,WIL,"Wilmington, Delaware",464,45,28.0,NaN
57243,2021,4,State Supported,Vermonter,Vermonter,57,WNL,"Windsor Locks, Connecticut",21,12,35.0,NaN
57244,2021,4,State Supported,Vermonter,Vermonter,57,WNM,"Windsor, Vermont",14,10,26.0,NaN


In [ ]:
# Hidden tests are within this cell

## 5.0 Check years, quarters covered

### 5.1 Fiscal years and quarters [1 pt]

Group `stations` on the fiscal year and fiscal quarter, and return a count of the number of rows
associated with each year-quarter combination. Sort the rows in descending order, reset the index,
and assign the new column the name "Rows." Assign the new `DataFrame` the name `periods`.

In [50]:
# YOUR CODE HERE
# periods = stations.groupby(["Fiscal Year", "Fiscal Quarter"])["Service"].count().sort_values(ascending = False, ignore_index = True).rename("Rows")

periods = stations.groupby(["Fiscal Year", "Fiscal Quarter"])["Service"].count().reset_index(name='Rows').sort_values(by='Rows', ascending=False).reset_index(drop=True)
periods

,Fiscal Year,Fiscal Quarter,Rows
0,2024,2,6519
1,2024,3,6219
2,2023,3,6137
3,2023,4,6112
4,2024,1,6059
5,2023,2,5819
6,2023,1,5682
7,2022,1,5656
8,2022,4,5530
9,2022,2,5484


In [ ]:
# Hidden tests are within this cell

### 5.2 Service lines [1 pt]

Group `stations` on the fiscal year, fiscal quarter, and service line and return a count of the number
of rows associated with each year-quarter-service line combination. Sort the rows in descending
order, reset the index, and assign the new column the name "Rows." Assign the new `DataFrame` the
name `periods`.

In [51]:
# periods = stations.groupby(["Fiscal Year", "Fiscal Quarter", "Service Line"])["Service"].count().sort_values(ascending = False, ignore_index = True).rename("Rows")
periods = stations.groupby(["Fiscal Year", "Fiscal Quarter", "Service Line"])["Service"].count().reset_index(name='Rows').sort_values(by='Rows', ascending=False).reset_index(drop=True)
periods

,Fiscal Year,Fiscal Quarter,Service Line,Rows
0,2023,4,State Supported,3001
1,2024,3,State Supported,2962
2,2023,3,State Supported,2929
3,2024,2,State Supported,2911
4,2024,1,State Supported,2863
5,2022,1,State Supported,2768
6,2023,2,State Supported,2729
7,2023,1,State Supported,2634
8,2022,4,State Supported,2611
9,2022,3,State Supported,2604


In [ ]:
# Hidden tests are within this cell

## 6.0 Persist data

### 6.1 Recheck data

In [46]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68412 entries, 17506 to 57245
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Fiscal Year                 68412 non-null  int16  
 1   Fiscal Quarter              68412 non-null  int8   
 2   Service Line                68412 non-null  string 
 3   Service                     68412 non-null  string 
 4   Sub Service                 68412 non-null  string 
 5   Train Number                68412 non-null  int16  
 6   Arrival Station Code        68412 non-null  string 
 7   Arrival Station Name        68412 non-null  string 
 8   Total Detraining Customers  68412 non-null  int32  
 9   Late Detraining Customers   68412 non-null  int32  
 10  Avg Min Late (Lt CS)        52705 non-null  float32
 11  Avg Min Late (Lt C)         4668 non-null   float32
dtypes: float32(2), int16(2), int32(2), int8(1), string(5)
memory usage: 4.5 MB


### 6.2 Write to file [1 pt]

Write data to a CSV file.

In [47]:
filepath = data_interim_path.joinpath("station_performance_metrics-v1p0.csv")
stations.to_csv(filepath, index=False)

In [ ]:
# Hidden tests are within this cell

## 6.0 Watermark

In [49]:
%load_ext watermark
%watermark -h -i -iv -m -v

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.26.0

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 6.5.0-1020-aws
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit

Hostname: d1d21d7847a4

pandas: 2.2.3
numpy : 2.1.3

